In [ ]:
# This is the notebook we will use to make COVID API calls and build the dataframe columns relevant to COVID

In [ ]:
import csv
import requests
import pandas as pd
import numpy as np
import urllib.request
import codecs

In [ ]:
base_url = 'http://coronavirusapi.com/getTimeSeries/'
state = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", 
         "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", 
         "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", 
         "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", 
         "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", 
         "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", 
         "WV", "WI", "WY"]
master_list = []

In [ ]:
# Because the dataset uses seconds from Unix Epoch date, function returns datetime, from floationg point value.
def epochConverter(num):
    days = num/3600/24
    date = pd.to_datetime('01-01-1970') + pd.DateOffset(days=days)
    return date

In [ ]:
# Outer for loop opens url and converts each call for state from bytestring to something readable.
for state_url in state:
    state_response = urllib.request.urlopen(base_url+state_url)
    csvfile = csv.reader(codecs.iterdecode(state_response, 'utf-8'))

    #inner loop iterates over each row in the returned .csv from API and appends in list
    for row in csvfile:
        
        #because the API is returning headers as well for each call, the if statement removes the header line.
        if row[0] != 'seconds_since_Epoch':
            master_list.append({'State': state_url,
                             'Date': epochConverter(float(row[0])),
                             'Tested': float(row[1]),
                             'Positive': float(row[2]),
                             'Deaths': float(row[3]),
                             })  

In [ ]:
#convert list of dictionaries into a dataframe
covid_state_df = pd.DataFrame(master_list)
covid_state_df

In [ ]:
# Create new dataframe with just the most recent values.  We can decide later if we want this or the dataframe above
# with all of the complete data from the API

recent_date_df = covid_state_df.groupby('State').max()
recent_date_df = recent_date_df.reset_index()
recent_date_df.head()

In [ ]:
recent_date_df["Mortality Rate"] = ((recent_date_df["Deaths"])/(recent_date_df["Positive"]))*100
recent_date_df.head()

recent_date_df.to_csv("Resources/covid_api_data.csv")